# It works with NPL 2023a but not the versions after

In [7]:
import os
import xarray as xa
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as mpcrs
from pylibs.utils import get_dates, setup_cmap
from pylibs.plot_utils import setupax_2dmap

In [16]:
start_date = 2024110506
end_date = 2024110512
cycle_int = 6

m2Tag = 'M2I3NXGAS.5.12.4'
m2FileTag = 'inst3_2d_gas_Nx'
plotVar = 'AODANA'
save = 1
quality = 600

dates = get_dates(start_date, end_date, cycle_int)

In [8]:
m2Url = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2'
savedir = '/glade/work/swei/projects/mmm.pace_aod/plots/merra2'
if not os.path.exists(savedir):
    os.makedirs(savedir)

In [4]:
cnlvs = [0., 0.05, 0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.5, 2.5]
clridx = [2, 4, 5, 7, 8, 9, 10, 12, 14, 15, 16, 17, 18]
tuplelst, clrmap = setup_cmap('precip3_16lev', cnlvs, clridx)
clr_norm = mpcrs.BoundaryNorm(cnlvs, len(clridx), extend='both')

In [17]:
for date in dates:
    y4 = date.strftime('%Y')
    m2 = date.strftime('%m')
    pdyStr = date.strftime('%Y%m%d')
    dateStr = date.strftime('%Y%m%d%H')
    m2File = f'{m2Url}/{m2Tag}/{y4}/{m2}/MERRA2_400.{m2FileTag}.{pdyStr}.nc4'

    ds = xa.open_dataset(m2File)
    pltds = ds.sel(time=date)

    fig, ax, gl = setupax_2dmap([1., 1., 1., 1.], 'Glb', ccrs.PlateCarree(), lbsize=12)
    cn = ax.contourf(pltds.lon, pltds.lat, pltds[plotVar],
                   levels=cnlvs, cmap=clrmap, norm=clr_norm, extend='both')
    plt.colorbar(cn, ax=ax, pad=0.05, fraction=0.025, ticks=cnlvs)
    ax.set_title(f'time={date}')
    figname = f'MERRA2.{plotVar}.{dateStr}.png'
    if save:
        fig.savefig(figname, dpi=quality)
        plt.close()